In [1]:
import os
cachedir = '/rscratch/tpang/kinshuk/cache'
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
os.environ["TRANSFORMERS_CACHE"] = cachedir
os.environ["HF_DATASETS_CACHE"]= cachedir
from model import get_model
from loader.layers import param_count
from loader.data_module import make_data_module
import json
import torch
import random
import logging
import argparse
import numpy as np
import transformers
from pathlib import Path
import torch.backends.mps
import torch.backends.cudnn
from torch.cuda import (
    max_memory_allocated,
    reset_peak_memory_stats,
    reset_max_memory_allocated,
    memory_allocated,
)
from loader.logger import get_logger
from transformers import set_seed
# from accelerate import Accelerator
from os.path import exists, join, isdir
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence

from transformers.utils.logging import (
    set_verbosity_error as transformers_vb_err,
)
from datasets.utils.logging import (
    set_verbosity_error as datasets_vb_err,
)
from transformers import Seq2SeqTrainer
from traineval.eval import eval_func
logger = logging.getLogger(__name__)
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
from llamaft import ModelArguments, DataArguments, TrainingArguments, GenerationArguments

/rscratch/tpang/kinshuk/anaconda3/envs/kinbert/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-03 03:20:20.413405: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 03:20:20.413439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 03:20:20.414840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 03:20:20.423192: I tensorflow/core/platform/cpu_feature_guard.cc:182] Thi

In [2]:
# Setting up the arguments

model_args = ModelArguments(
    model_name_or_path="meta-llama/Llama-2-7b-hf"
)

data_args = DataArguments(
    eval_dataset_size=1024,
    max_eval_samples=1000,
    source_max_len = 1024,
    target_max_len = 256,

    dataset="alpaca", # DATASET [alpaca|chip2|self-instruct|hh-rlhf|oasst1|longform]
)

training_args = TrainingArguments(
    output_dir="./output",
    data_seed=7,
    save_strategy="steps",
    evaluation_strategy="steps",
    logging_strategy="steps",
    do_eval=True,
    eval_steps=187,
    adam_beta2=0.999,

    learning_rate=2e-6,     # LEARNING RATE
    
    max_steps=2000,         # NUMBER OF STEPS

    sortby="alpha",         # CAN DO "alpha" or "lora"

    num_layers=4,           # NUMBER OF LAYERS FOR FULL FINE-TUNING

    per_device_train_batch_size = 2, # BATCH-SIZE
    memlog=False,
)

generation_args = GenerationArguments(
    max_new_tokens=128 # default is 256
)

# If you need to use GenerationConfig or similar for generation_args
training_args.generation_config = transformers.GenerationConfig(
    **vars(generation_args)
)

# Combine arguments into a single Namespace object (if needed)
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args),
)

# Control randomness
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
set_seed(args.seed)  # transformers seed

In [3]:
if 'lora' in args.sortby:
    args.num_layers = 0
logger = logging.getLogger(__name__)
gpus = torch.cuda.device_count()
mempath = (
    f"/rscratch/tpang/kinshuk/RpMKin/llama_ft/{args.dataset}/"
    + f"{args.sortby}"
)
start_memory = [0] * gpus
end_memory = [0] * gpus
peek_memory = 0

if args.verbose:
        task_info = (
            f"\nSeed: {args.seed}\n"
            + f"Dataset: {args.dataset}\n"
            + f"Sort by: {args.sortby}\n"
            + f"Layers to train: {args.num_layers}\n"
        )
        print(task_info)
else:
    datasets_vb_err()
    transformers_vb_err()
    global _tqdm_active
    _tqdm_active = False


Seed: 7
Dataset: alpaca
Sort by: alpha
Layers to train: 4



In [4]:
def memall(gpus=gpus):
    for i in range(gpus):
        start_memory[i] = torch.cuda.memory_allocated(i)
    return sum(start_memory)

model, tokenizer = get_model(args)

for device in range(gpus):
    reset_peak_memory_stats(device=device)
    reset_max_memory_allocated(device=device)

weight_memory = memall()

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.43s/it]


Adding special tokens.
Sorted by  alpha
Training layers: ['model.layers.21.self_attn.v_proj', 'model.layers.22.self_attn.v_proj', 'model.layers.16.self_attn.v_proj', 'model.layers.19.self_attn.v_proj']
Enabling model.layers.16.self_attn.v_proj.weight parameter
Enabling model.layers.19.self_attn.v_proj.weight parameter
Enabling model.layers.21.self_attn.v_proj.weight parameter
Enabling model.layers.22.self_attn.v_proj.weight parameter


In [5]:
data_module = make_data_module(tokenizer=tokenizer, args=args) # type: ignore

Splitting train dataset in train and validation according to `eval_dataset_size`


In [6]:
dataset = {k:v for k,v in data_module.items()}
dataset

{'train_dataset': Dataset({
     features: ['input', 'output', 'length'],
     num_rows: 50978
 }),
 'eval_dataset': Dataset({
     features: ['input', 'output', 'length'],
     num_rows: 1000
 }),
 'predict_dataset': None,
 'data_collator': DataCollatorForCausalLM(tokenizer=LlamaTokenizer(name_or_path='meta-llama/Llama-2-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[PAD]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False), source_max_len=1024, target_max_len=256, train_on_source=False, predict_with_generate=False)}

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset['train_dataset'], # type: ignore
    batch_size=args.per_device_train_batch_size,
    collate_fn=dataset['data_collator'],
    shuffle=True,
)

input_memory = memall()- weight_memory

train_dataloader.__len__()

25489

In [8]:
def loss_fn(x, y):
    "A Flat CrossEntropy" 
    return torch.nn.functional.cross_entropy(x.view(-1, x.shape[-1]), y.view(-1))

from tqdm.auto import tqdm
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
train_losses = []
val_losses = []
val_accs = []

model.train()
for epoch in range(1):
        train_loss = 0
        tr_steps = 0
        for step, batch in enumerate(tqdm(train_dataloader)):
            optimizer.zero_grad()
            batch = {k: v.to(model.device) for k, v in batch.items()}
            output = model(**batch)
            activation_memory = memall() - weight_memory
            # loss = loss_fn(out.logits, batch["labels"]) / args.gradient_accumulation_steps
            loss = output.loss
            train_loss += loss.item()
            loss.backward()
            gradient_memory = memall() - weight_memory
            optimizer.step()
            optimizer_memory = memall() - gradient_memory - weight_memory 
            tr_steps += 1
            train_losses.append(train_loss/tr_steps)
            if step % 50 == 0:
                print(f'Step: {step}, Train Loss: {train_loss/tr_steps}')
            if step == args.max_steps:
                model.eval()
                break

total_memory = memall()

  0%|          | 2/25489 [00:01<3:39:12,  1.94it/s]

Step: 0, Train Loss: 1.2829551696777344


  0%|          | 51/25489 [00:11<1:23:37,  5.07it/s]

Step: 50, Train Loss: 1.633158242001253


  0%|          | 102/25489 [00:22<1:30:09,  4.69it/s]

...

Step: 1900, Train Loss: 1.215531432804216


  8%|▊         | 1951/25489 [07:12<1:24:35,  4.64it/s]

Step: 1950, Train Loss: 1.2118963566133758


  8%|▊         | 2000/25489 [07:23<1:26:42,  4.51it/s]

Step: 2000, Train Loss: 1.2110853766885654


In [9]:
optimizer.zero_grad()
model.eval()
trainer=Seq2SeqTrainer(
                model=model,
                tokenizer=tokenizer,
                args=training_args,
                **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
            )
all_metrics = {"run_name": args.run_name}
if args.do_eval:
    all_metrics = eval_func(args, logger, trainer, all_metrics)
total_memory = memall()
peek_memory = max([max_memory_allocated(i) for i in range(gpus)])


INFO:__main__:*** Evaluate ***


***** eval metrics *****
  eval_loss               =     1.1706
  eval_runtime            = 0:01:07.61
  eval_samples_per_second =     14.791
  eval_steps_per_second   =      1.849


In [10]:
all_metrics

{'run_name': './output',
 'eval_loss': 1.1705690622329712,
 'eval_runtime': 67.611,
 'eval_samples_per_second': 14.791,
 'eval_steps_per_second': 1.849}

In [11]:
param_count(model)
memory_string = (
    f"Method           : {args.sortby}\n"
    f"Layers           : {args.num_layers}\n"
    f"Learning Rate    : {args.learning_rate}\n"
    f"Batch size       : {args.per_device_train_batch_size}\n"
    f"Weight memory    : {weight_memory / 1e6} MB\n"
    f"Activation memory: {activation_memory / 1e6} MB\n"
    f"Gradient memory  : {gradient_memory / 1e6} MB\n"
    f"Optimizer memory : {optimizer_memory / 1e6} MB\n"
    f"Total memory     : {total_memory / 1e6} MB\n"
    f"Peak memory      : {peek_memory / 1e6} MB\n"
)
print(memory_string)

Total params: 6738.42M, Trainable: 198.18M
Method           : alpha
Layers           : 4
Learning Rate    : 2e-06
Batch size       : 2
Weight memory    : 27087.929344 MB
Activation memory: 3427.019264 MB
Gradient memory  : 2445.723648 MB
Optimizer memory : 0.0 MB
Total memory     : 28740.913152 MB
Peak memory      : 37007.917056 MB



In [12]:
# SAVE TRAINING HISTORY
base = {"train_loss": train_loss,}
savepath = f"./output/{args.dataset}/lr_{args.learning_rate}/batch_{args.per_device_train_batch_size}/{args.sortby}/layers_{args.num_layers}"
if True:
    Path(savepath).mkdir(parents=True, exist_ok=True)
    np.save(os.path.join(savepath, "finetune.npy"), base) # type: ignore
    with open(os.path.join(savepath, "metrics.json"), "w") as fout:
        fout.write(json.dumps(all_metrics))
    log_info = (
        f"\n\n{args.dataset} "
        + f"Batch Size {args.per_device_train_batch_size} "
        + f"{args.sortby} fine-tuning "
        + f"{args.num_layers} Layers"
    )
    logger = get_logger(savepath, "memlog.log")
    logger.info(log_info)
    logger.info(f"\n{memory_string}\n")
    if (args.do_train or args.do_eval or args.do_predict):
        with open(os.path.join(savepath, "metrics.json"), "w") as fout:
            fout.write(json.dumps(all_metrics))

INFO:root:

alpaca Batch Size 2 alpha fine-tuning 4 Layers


2024-04-03 03:29:40;INFO;

alpaca Batch Size 2 alpha fine-tuning 4 Layers


INFO:root:
Method           : alpha
Layers           : 4
Learning Rate    : 2e-06
Batch size       : 2
Weight memory    : 27087.929344 MB
Activation memory: 3427.019264 MB
Gradient memory  : 2445.723648 MB
Optimizer memory : 0.0 MB
Total memory     : 28740.913152 MB
Peak memory      : 37007.917056 MB




2024-04-03 03:29:40;INFO;
Method           : alpha
Layers           : 4
Learning Rate    : 2e-06
Batch size       : 2
Weight memory    : 27087.929344 MB
Activation memory: 3427.019264 MB
Gradient memory  : 2445.723648 MB
Optimizer memory : 0.0 MB
Total memory     : 28740.913152 MB
Peak memory      : 37007.917056 MB


